In [1]:
import os, sys
import attr
# add to path
sys.path.append("../..")

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from PIL import Image
seaborn.set()

In [3]:

from filterflow.smc import SMC
from filterflow.base import State, Observation, InputsBase, StateSeries, DTYPE_TO_OBSERVATION_SERIES

from filterflow.observation.linear import LinearObservationModel
from filterflow.observation.base import ObservationSampler

from filterflow.transition.random_walk import RandomWalkModel
from filterflow.proposal.base import BootstrapProposalModel

from filterflow.resampling.criterion import NeffCriterion, AlwaysResample
from filterflow.resampling.standard.systematic import SystematicResampler
from filterflow.resampling.standard.multinomial import MultinomialResampler

from filterflow.resampling.base import NoResampling

from filterflow.state_space_model import StateSpaceModel

# Define State Space Model

## Specify linear dynamic

In [4]:
observation_dim = 2
state_dim = 2
T = 150
initial_state          = np.repeat(0., state_dim)
transition_matrix      = np.eye(state_dim) * 0.5
transition_covariance  = np.eye(state_dim) * 0.5
observation_matrix     = np.eye(observation_dim) *0.5
observation_covariance = np.eye(observation_dim)*0.1

In [5]:
tf_observation_matrix = tf.constant(observation_matrix, dtype = tf.float32)
tf_observation_covariance = tf.constant(observation_covariance, dtype = tf.float32)


tf_transition_matrix = tf.constant(transition_matrix, dtype = tf.float32)
tf_transition_covariance =  tf.constant(transition_covariance, dtype = tf.float32)

## Observation Model + Sampler

In [6]:
class GaussianObservations(ObservationSampler):
    
    def __init__(self, observation_matrix, observation_covariance):
        
        obs_dim = observation_matrix.shape.as_list()[0]
        self._observation_error = tfd.MultivariateNormalTriL(tf.constant(np.repeat(0., obs_dim), dtype = tf.float32),
                                                            observation_covariance)
    
        self._observation_matrix = observation_matrix

    def loglikelihood(self, state: State, observation: Observation):
        """Computes the loglikelihood of an observation given proposed particles
        :param state: State
            Proposed (predicted) state of the filter given State at t-1 and Observation
        :param observation: Observation
            User/Process given observation
        :return: a tensor of loglikelihoods for all particles
        :rtype: tf.Tensor
        """
        error = observation.observation - tf.linalg.matvec(self._observation_matrix, state.particles)
        return self._observation_error.log_prob(error)
    
    def sample(self, state: State, inputs: InputsBase=None):
        """Samples a new proposed state conditionally on prior state and some inputs
        :param state: State
            State of the filter at t-1
        :param inputs: InputsBase
            Input for transition model
        :return: sampled Observation
        :rtype: Observation
        """
        pushed_particles = tf.linalg.matvec(self._observation_matrix, state.particles)
        res = pushed_particles + self._observation_error.sample([state.batch_size, state.n_particles])
        return Observation(res)
        


In [7]:
observation_model = GaussianObservations(tf_observation_matrix, tf_observation_covariance)

## Transition

In [8]:
state_dim = tf_transition_covariance.shape.as_list()[0]
transition_noise = tfd.MultivariateNormalTriL(tf.constant(np.repeat(0., state_dim), dtype=tf.float32), 
                                                                    tf_transition_covariance)

transition_model = RandomWalkModel(tf_transition_matrix, transition_noise)

## State Space

In [9]:
ss_model = StateSpaceModel(observation_model=observation_model, transition_model=transition_model)

In [10]:
initial_particle = tf.constant(np.random.random(2), dtype = tf.float32)

In [11]:
states, observations = ss_model(initial_particle, T)

In [12]:
np_obs = np.array([observations.read(t).observation.numpy().squeeze() for t in range(T)])
np_states = np.array([states.read(t).particles.numpy().squeeze() for t in range(T)])


linspace  = np.arange(np_obs[:,0].shape[0])
fig, axes = plt.subplots(nrows=2, figsize=(10, 5))
axes[0].scatter(linspace, np_obs[:,0], marker='o', color='g', label='observations')
axes[0].plot(np_states[:, 0], color='b', label='$x^1_t$')
axes[1].plot(np_states[:, 1], color='b', label='$x^2_t$')
axes[1].scatter(linspace, np_obs[:,1], marker='o', color='g', label='observations')
_ = axes[0].legend(loc='upper left')
_ = axes[1].legend(loc='upper left')
_ = fig.suptitle('Latent space and observations', fontsize=15)

AttributeError: 'numpy.ndarray' object has no attribute 'read'

## Specify Particle Filter Components

In [ ]:
# store observations
batch_size = 1
n_particles = 1000
dimension = 2

In [ ]:
proposal_model = BootstrapProposalModel(transition_model)

In [ ]:
resampling_criterion = AlwaysResample()
resampling_method = MultinomialResampler()

In [ ]:
particle_filter = SMC(observation_model, 
                                 transition_model, proposal_model, resampling_criterion, resampling_method)

In [ ]:
import time

In [ ]:

weights = tf.ones((batch_size, n_particles), dtype=float)/tf.cast(n_particles, float)
initial_particles = tf.random.uniform((batch_size, n_particles, dimension), -1, 1)
log_likelihoods = tf.zeros((batch_size), dtype=float)
initial_state = State(particles=initial_particles, log_weights= tf.math.log(weights), weights=weights, log_likelihoods=log_likelihoods)

In [ ]:
record_states = particle_filter(initial_state, observations)

In [ ]:
# compute states
computed_states = []
for t in range(observations.size()):
    state = record_states.read(t)
    log_weights = state.log_weights.numpy()
    particles = state.particles.numpy()[:,:,0]
    weights = np.exp(log_weights)
    computed_state = np.sum((weights * particles), axis=1)
    computed_states.append(computed_state)
    
computed_states = np.array(computed_states).flatten()

## Plot

In [ ]:
import numpy as np
import bqplot.pyplot as bplt
from bqplot import LinearScale, Axis, Lines, Bars, Figure
axes_options = {'x': {'label': 't'}, 'y': {'label': 'y'}}
fig = bplt.figure(animation_duration=0)
lines = bplt.plot(
                x=np.linspace(0, len(np_obs), len(np_obs)), 
                y=np_obs[:,0], 
                colors=['red'], 
                axes_options=axes_options)

if False:
    true_state_ln = bplt.plot(
                x=np.linspace(0, len(np_obs), len(np_obs)), 
                y=np_states[:,0], 
                colors=['blue'], 
                axes_options=axes_options)

computed_lin = bplt.plot(
                x=np.linspace(0, len(np_obs), len(np_obs)), 
                y=computed_states, 
                colors=['green'], 
                axes_options=axes_options)

scatt = bplt.scatter([], [], colors=['blue'], axes_options=axes_options)


max_y = np.max([np_obs[:,0]])
min_y = np.min([computed_states, np_obs[:,0]])

bplt.set_lim(min_y-0.5, max_y+0.5, 'y')
bplt.set_lim(0, len(np_obs), 'x')

fig

In [ ]:
jitter = np.random.normal(0., 0.25, n_particles)

gif_dir = './gif_dir'
os.mkdir(gif_dir)
for t in range(observation_series.size()):
    state = record_states.read(t)
    log_weights = state.log_weights.numpy()
    particles = state.particles.numpy()[0,:,0]
    weights = np.exp(log_weights)
    x = t + jitter
    y = particles
    
    plt.plot(np.linspace(0, len(ghmm_output.observations), len(ghmm_output.observations)), 
                     ghmm_output.observations[:,0], 'rx')

    computed_lin = plt.plot(np.linspace(0, t, t), 
                            computed_states[:t], 'g')
    plt.ylim(-2.,2.)
    plt.plot(x, y, 'bx')
    plt.savefig(os.path.join(gif_dir,'plot_{0}.png'.format(t)))
    plt.clf()
    

In [ ]:
fps = [os.path.join('./gif_dir', 'plot_{0}.png'.format(t)) for t in range(T+1)]

frames = [Image.open(fn) for fn in fps]

frames[0].save('pf.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)

## Likelihood

In [ ]:
final_state = record_states.read(150)

In [ ]:
final_state.log_likelihoods